In [5]:
import nest_asyncio
nest_asyncio.apply()

from hddCRP.simulations import simulate_sessions
from hddCRP.modelBuilder import cdCRP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import arviz as az

import os
import sys

In [6]:



args = [2, 1, 5]
# if __name__ == "__main__":
#     for ii, arg in enumerate(sys.argv):
#         if(ii < len(args)):
#             args[ii] = arg

simulation_id = args[0];
min_blocks = args[1];
max_blocks = args[2];

run_range = range(0, 20)
results_directory = "Results/Simulations/"


if(simulation_id == 0):
    alpha = 2
    different_context_weights = [0.75, 0.75];
    within_session_timescales  = {"A" : 20}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.75

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 * n_blocks];
    num_subjects = lambda n_blocks : 1;
    num_responses = 3
elif(simulation_id == 1):
    alpha = 8
    different_context_weights = [0.25, 0.25];
    within_session_timescales  = {"A" : 40}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.25

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 * n_blocks];
    num_subjects = lambda n_blocks : 1;
    num_responses = 3
elif(simulation_id == 2):
    alpha = 2
    different_context_weights = [0.75, 0.75];
    within_session_timescales  = {"A" : 20}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.75

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 ];
    num_subjects = lambda n_blocks : n_blocks;
    num_responses = 3
elif(simulation_id == 3):
    alpha = 8
    different_context_weights = [0.25, 0.25];
    within_session_timescales  = {"A" : 40}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.25

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 ];
    num_subjects = lambda n_blocks :  n_blocks;
    num_responses = 3
else:
    raise NotImplementedError("No sim found")

block_range = range(min_blocks, max_blocks+1)

if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)



NotImplementedError: No sim found

In [ ]:



for block_idx in block_range:
    print(f"BLOCK {block_idx}")
    fit_file = f"{results_directory}/sim_{simulation_id}_block_{block_idx}.pkl"
    fit_summary_file = f"{results_directory}/sim_summary_{simulation_id}_block_{block_idx}.pkl"
    if((not os.path.isfile(fit_file)) or (not os.path.isfile(fit_summary_file))):
        simulation_fits = pd.DataFrame()
        simulation_fit_metrics = pd.DataFrame()
        for run_idx in run_range:
            print(f"BLOCK {block_idx} - RUN {run_idx}")
            sim_seed  = (simulation_id+1) * 10000 + run_idx*100
            stan_seed = (simulation_id+1) * 10000 + run_idx*100 + block_idx
            sim_rng = np.random.Generator(np.random.MT19937(sim_seed))
            
            seqs = [];
            subject_labels = [];
            session_labels_all = [];
            for jj in range(num_subjects(block_idx)):
                seqs_c = simulate_sessions(session_lengths=session_lengths(block_idx), session_labels=session_labels(block_idx), num_responses=num_responses, 
                                        alpha=alpha,
                                        different_context_weights=different_context_weights,
                                        within_session_timescales=within_session_timescales, between_session_timescales=between_session_timescales,
                                        repeat_bias_1_back=repeat_bias_1_back, rng=sim_rng)
                subject_labels += [jj] * len(seqs_c)
                session_labels_all += session_labels(block_idx)
                seqs += seqs_c;

            model = cdCRP(seqs, subject_labels=subject_labels, session_labels=session_labels_all);
            model.build(random_seed=stan_seed);
            model.fit_model()

            map_fit = model.get_map()
            fit_df = model.fit.to_frame()
            fit_df["block"] = block_idx
            fit_df["run"]   = run_idx
            summary_df = model.fit_summary()
            summary_df["block"] = block_idx
            summary_df["run"]   = run_idx
            summary_df["MAP"] = pd.Series(map_fit)

            simulation_fit_metrics = pd.concat([simulation_fit_metrics,summary_df], copy=False)
            simulation_fits = pd.concat([simulation_fits,fit_df], copy=False)

        simulation_fits.to_pickle(fit_file)
        simulation_fit_metrics.to_pickle(fit_summary_file)
    else:
        print("Fit files found: not overriding")

BLOCK 1
Fit files found: not overriding
BLOCK 2
BLOCK 2 - RUN 0
Building...


Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  

BLOCK 2 - RUN 1
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  

BLOCK 2 - RUN 2
Building...


...


Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  

BLOCK 5 - RUN 18
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  

BLOCK 5 - RUN 19
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  

In [ ]:
simulation_fits

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha,timeconstant_within_session_A,repeat_bias_1_back,context_similarity_depth_1,context_similarity_depth_2,block,run
draws,,,,,,,,,,,,,,
0,-267.119347,0.921425,0.436410,3.0,7.0,0.0,269.896135,6.890105,88.210453,0.611146,0.055295,0.392704,5,0
1,-267.274289,0.997466,0.349248,3.0,7.0,0.0,267.927646,7.481635,53.143108,0.542713,0.206761,0.184612,5,0
2,-267.662932,0.974882,0.465194,3.0,7.0,0.0,269.363044,5.101883,67.032126,0.590533,0.175776,0.629755,5,0
3,-267.330095,0.824193,0.425666,3.0,7.0,0.0,270.930841,4.539853,90.490734,0.594171,0.021856,0.349415,5,0
4,-269.060779,0.912094,0.436410,3.0,15.0,0.0,271.085871,2.779941,85.030865,0.564466,0.229318,0.397717,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-273.600976,0.997650,0.433359,3.0,7.0,0.0,278.627445,5.180791,3.092179,0.340442,0.317702,0.194060,5,19
3996,-272.546672,0.932823,0.577359,3.0,7.0,0.0,273.237516,12.702850,30.396577,0.367122,0.068198,0.162249,5,19
3997,-272.239866,0.999053,0.442733,3.0,7.0,0.0,273.136094,5.726084,21.483972,0.386342,0.055463,0.406772,5,19
